## Analysis of tangling across layers (panel E)

In [ ]:
%matplotlib inline
import numpy as np
from functions_notebook import get_layers, measure_tangling
from definitions import ROOT_DIR
import os
import pickle
import umap
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from envs.environment_factory import EnvironmentFactory
import matplotlib
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import VecNormalize
from functions_notebook import make_parallel_envs
from myosuite.envs.myo.myochallenge.baoding_v1 import Task


In [ ]:
# df1 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_50_episodes_activations_info_ccw", "data.hdf"))
# df2 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_50_episodes_activations_info_cw", "data.hdf"))

df1 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_500_episodes_activations_info_small_variations_ccw", "data.hdf"))
df2 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_500_episodes_activations_info_small_variations_cw", "data.hdf"))

# df1 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_50_episodes_activations_info_small_variations_ccw", "data.hdf"))
# df2 = pd.read_hdf(os.path.join(ROOT_DIR, "data", "rollouts", "final_model_50_episodes_activations_info_small_variations_cw", "data.hdf"))

df = pd.concat((df1, df2)).reset_index()
df.keys()

In [ ]:
def average_by_timestep(vec, timesteps):
    out_vec = []
    for ts in sorted(np.unique(timesteps)):
        out_vec.append(np.mean(vec[timesteps == ts], axis=0))
    return np.vstack(out_vec)


In [ ]:
n_comp = 25
layer_names = ["observation", "lstm_state_1", "lstm_out", "layer_1_out", "layer_2_out", "action"]
readable_layer_names = ["Observation", "LSTM state", "LSTM out", "Layer 1 out", "Layer 2 out", "Action"]

pca = PCA(n_components=n_comp)

for layer in layer_names:
    print("Layer: ", layer)
    data = np.array(df[layer].to_list())
    embeddings = pca.fit_transform(data)
    df[layer + "_pc"] = list(embeddings)

In [ ]:
n_comp = 3
first_step = 0
df_trunc = df[df.step > first_step]
for layer in layer_names:
    data = np.array(df_trunc[layer + "_pc"].to_list())
    out_cw = data[df_trunc.task == "cw"]
    out_ccw = data[df_trunc.task == "ccw"]
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(projection="3d")

    cmap = matplotlib.colormaps["Reds"]
    color_list = [cmap(i) for i in np.linspace(0.5, 1, len(df_trunc.step[df_trunc.task == "cw"].unique()))]    
    colors = [color_list[idx] for idx in (df_trunc.step[df_trunc.task == "cw"] - first_step - 1)]
    # ax.scatter(out_cw[:, 0], out_cw[:, 1], out_cw[:, 2], alpha=0.1, s=0.2, c=colors)
    mean_traj = average_by_timestep(out_cw[:, :3], df_trunc.step[df_trunc.task == "cw"])
    # ax.plot(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], color="r", linewidth=1)
    ax.scatter(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], c=color_list, label="Clockwise")

    cmap = matplotlib.colormaps["Blues"]
    color_list = [cmap(i) for i in np.linspace(0.5, 1, len(df_trunc.step[df_trunc.task == "ccw"].unique()))]    
    colors = [color_list[idx] for idx in (df_trunc.step[df_trunc.task == "ccw"] - first_step - 1)]
    # ax.scatter(out_ccw[:, 0], out_ccw[:, 1], out_ccw[:, 2], alpha=0.1, s=0.2, c=colors)
    mean_traj = average_by_timestep(out_ccw[:, :3], df_trunc.step[df_trunc.task == "ccw"])
    # ax.plot(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], color="b", linewidth=1)
    ax.scatter(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], c=color_list, label="Counter-clockwise")
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    # ax.set_xlim((-0.8, 0.8))
    # ax.set_ylim((-.8, .8))
    # ax.set_zlim((-.8, .8))
    # ax.set_title(title + " - high variance PCs")
    # ax.view_init(25, 47)
    ax.set_xlabel("\nPC 1")
    ax.set_ylabel("\nPC 2")
    ax.set_zlabel("\nPC 3")
    # ax.legend()
    ax.set_box_aspect(aspect=None, zoom=0.75)

    # fig = plt.figure()
    # ax = plt.axes(projection='3d')
    # ax.scatter(data[:, 0], data[:, 1], data[:, 2], alpha=0.2, s=1, c=df["step"])
    # plt.title(layer)
    # plt.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_5", f"pca_{layer}.png"), format="png", dpi=600, bbox_inches="tight")
    plt.show()

In [ ]:
# Graph per layer with UMAP
n_comp = 3
first_step = 13
random_state = 0
reducer = umap.UMAP(n_components=n_comp, random_state=random_state)
df_trunc = df[df.step > first_step]
for layer in layer_names:
    print("Layer", layer)
    data = np.array(df_trunc[layer + "_pc"].to_list())
    data = reducer.fit_transform(data)
    out_cw = data[df_trunc.task == "cw"]
    out_ccw = data[df_trunc.task == "ccw"]
    out_dir = os.path.join(ROOT_DIR, "data", "embeddings")
    
    with open(os.path.join(out_dir, f"umap_embeddings_ccw_{layer}.pkl"), "wb") as file:
        pickle.dump(out_ccw, file)
    with open(os.path.join(out_dir, f"umap_embeddings_cw_{layer}.pkl"), "wb") as file:
        pickle.dump(out_cw, file)
        
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(projection="3d")

    cmap = matplotlib.colormaps["Reds"]
    color_list = [cmap(i) for i in np.linspace(0.3, 1, len(df_trunc.step[df_trunc.task == "cw"].unique()))]    
    colors = [color_list[idx] for idx in (df_trunc.step[df_trunc.task == "cw"] - first_step - 1)]
    ax.scatter(out_cw[:, 0], out_cw[:, 1], out_cw[:, 2], alpha=0.1, s=1, c=colors)
    mean_traj = average_by_timestep(out_cw[:, :3], df_trunc.step[df_trunc.task == "cw"])
    # ax.plot(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], color="r", linewidth=1)
    # ax.scatter(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], c=color_list)

    cmap = matplotlib.colormaps["Blues"]
    color_list = [cmap(i) for i in np.linspace(0.3, 1, len(df_trunc.step[df_trunc.task == "ccw"].unique()))]    
    colors = [color_list[idx] for idx in (df_trunc.step[df_trunc.task == "ccw"] - first_step - 1)]
    ax.scatter(out_ccw[:, 0], out_ccw[:, 1], out_ccw[:, 2], alpha=0.1, s=1, c=colors)
    mean_traj = average_by_timestep(out_ccw[:, :3], df_trunc.step[df_trunc.task == "ccw"])
    # ax.plot(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], color="b", linewidth=1)
    # ax.scatter(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], c=color_list)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    # ax.set_xlim((-0.8, 0.8))
    # ax.set_ylim((-.8, .8))
    # ax.set_zlim((-.8, .8))
    # ax.set_title(title + " - high variance PCs")
    # ax.view_init(25, 47)
    ax.set_xlabel("\nUMAP 1", fontsize=12)
    ax.set_ylabel("\nUMAP 2", fontsize=12)
    ax.set_zlabel("\nUMAP 3", fontsize=12)
    ax.set_box_aspect(aspect=None, zoom=0.75)
    ax.ticklabel_format(style="sci", scilimits=(-2, 2))
    ax.locator_params(axis='both', nbins=4)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=10)
    # fig = plt.figure()
    # ax = plt.axes(projection='3d')
    # ax.scatter(data[:, 0], data[:, 1], data[:, 2], alpha=0.2, s=1, c=df["step"])
    # plt.title(layer)
    plt.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_5", f"umap_{layer}_1000_episodes_from_step_{first_step}_seed_{random_state}.png"), format="png", dpi=600, bbox_inches="tight")
    plt.show()

In [ ]:
def measure_tangling(data):
    derivative = np.gradient(data,axis=0) * 40  # sample frequency

    # epsilon = 0.1*np.mean(np.linalg.norm(data,axis=1))
    epsilon = 1e-10 # * np.mean(np.linalg.norm(data, axis=1))
    # epsilon = 1e-1

    Q_all = []
    for t in range(derivative.shape[0]):
        Q = (np.linalg.norm(derivative[t] - derivative, axis=1)**2) / (epsilon + np.linalg.norm(data[t] - data, axis=1)**2)
        Q = np.max(Q)
        Q_all.append(Q)
    
    return np.mean(Q_all)  # as per definition

In [ ]:
num_components = 8

for layer in layer_names:
    tangling_list = []
    episode_pc_list = df.groupby(["episode", "task"])[layer + "_pc"].agg(lambda x: np.vstack(x)[:, :num_components]).tolist()
    for episode_pc in episode_pc_list:
        tangling_list.append(measure_tangling(episode_pc))
    print(layer, np.mean(tangling_list))

In [ ]:
# Scatter plot tangling memory vs observation and memory vs action
scatter_layer_names = ["observation", "lstm_state_1", "action"]
tangling_dict = {}
for layer in scatter_layer_names:
    tangling_list = []
    episode_pc_list = df.groupby(["episode", "task"])[layer + "_pc"].agg(lambda x: np.vstack(x)[:, :num_components]).tolist()
    for episode_pc in episode_pc_list:
        tangling_list.append(measure_tangling(episode_pc))
    tangling_dict[layer] = tangling_list

# memory vs observation
lim = (0, 8500)
fig, ax = plt.subplots(figsize=(3.6, 3.6))
ax.scatter(tangling_dict["observation"], tangling_dict["lstm_state_1"], s=0.5, color="dodgerblue")
ax.plot(lim, lim, color="red")
ax.set_xlabel("Observation space (86D)", fontsize=13)
ax.set_ylabel("Memory layer\nrepresentation (256D)", fontsize=13)
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.ticklabel_format(scilimits=(3, 3))
out_name = "scatter_tangling_memory_vs_observation.png"
fig.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_2", out_name), format="png", dpi=800, bbox_inches="tight")
fig.show()

# memory vs action
lim = (0, 12500)
fig, ax = plt.subplots(figsize=(3.6, 3.6))
ax.scatter(tangling_dict["action"], tangling_dict["lstm_state_1"], s=0.5, color="dodgerblue")
ax.plot(lim, lim, color="red")
ax.set_xlabel("Action space (39D)", fontsize=13)
ax.set_ylabel("Memory layer\nrepresentation (256D)", fontsize=13)
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.ticklabel_format(scilimits=(3, 3))
out_name = "scatter_tangling_memory_vs_action.png"
fig.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_2", out_name), format="png", dpi=800, bbox_inches="tight")
fig.show()
        

In [ ]:
for map_name in ["Reds", "Blues"]:
    cmap = matplotlib.colormaps[map_name]
    color_list = [cmap(i) for i in np.linspace(0.5, 1, 200)]    
    c_mat = np.array(color_list)
    c_mat = np.dstack([c_mat for _ in range(30)]).transpose(2, 0, 1)
    plt.imshow(c_mat)
    plt.axis("off")
    plt.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_5", f"cmap_{map_name}.png"), format="png", dpi=600, bbox_inches="tight")
    plt.show()

In [ ]:
len_ep1 = len(layers[0])
len_ep2 = len(layers[1])
len_ep3 = len(layers[2])

layers = [item for sublist in layers for item in sublist] # Concatenate all episodes

n_comp = 3
umap_apply = umap.UMAP(n_components=n_comp,random_state=42)

obs = np.array([d['observation'] for d in layers])
obs_trans = umap_apply.fit_transform(obs)
q_obs_ep1 = measure_tangling(obs_trans[0:len_ep1])
q_obs_ep2 = measure_tangling(obs_trans[len_ep1:len_ep1+len_ep2])
q_obs_ep3 = measure_tangling(obs_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_obs = np.mean(np.mean(q_obs_ep1)+np.mean(q_obs_ep2)+np.mean(q_obs_ep3))

lstm = np.array([d['LSTM hidden state'] for d in layers])
lstm_trans = umap_apply.fit_transform(lstm)
q_lstm_ep1 = measure_tangling(lstm_trans[0:len_ep1])
q_lstm_ep2 = measure_tangling(lstm_trans[len_ep1:len_ep1+len_ep2])
q_lstm_ep3 = measure_tangling(lstm_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_lstm = np.mean(np.mean(q_lstm_ep1)+np.mean(q_lstm_ep2)+np.mean(q_lstm_ep3))

l1 = np.array([d['Linear layer 1'] for d in layers])
l1_trans = umap_apply.fit_transform(l1)
q_l1_ep1 = measure_tangling(l1_trans[0:len_ep1])
q_l1_ep2 = measure_tangling(l1_trans[len_ep1:len_ep1+len_ep2])
q_l1_ep3 = measure_tangling(l1_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_l1 = np.mean(np.mean(q_l1_ep1)+np.mean(q_l1_ep2)+np.mean(q_l1_ep3))

l2 = np.array([d['Linear layer 2'] for d in layers])
l2_trans = umap_apply.fit_transform(l2)
q_l2_ep1 = measure_tangling(l2_trans[0:len_ep1])
q_l2_ep2 = measure_tangling(l2_trans[len_ep1:len_ep1+len_ep2])
q_l2_ep3 = measure_tangling(l2_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_l2 = np.mean(np.mean(q_l2_ep1)+np.mean(q_l2_ep2)+np.mean(q_l2_ep3))

acts = np.array([d['actions'] for d in layers])
acts_trans = umap_apply.fit_transform(acts)
q_acts_ep1 = measure_tangling(acts_trans[0:len_ep1])
q_acts_ep2 = measure_tangling(acts_trans[len_ep1:len_ep1+len_ep2])
q_acts_ep3 = measure_tangling(acts_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_acts = np.mean(np.mean(q_acts_ep1)+np.mean(q_acts_ep2)+np.mean(q_acts_ep3))

4. Plot the projected trajectories

In [ ]:
projected_all = [obs_trans,lstm_trans,l1_trans,l2_trans,acts_trans]
filenames = ['obs','lstm','l1','l2','acts']
titles = ['Observations','LSTM','Linear 1','Linear 2','Actions']
Qs = [Q_obs,Q_lstm,Q_l1,Q_l2,Q_acts]

for projected,filename,title,q in zip(projected_all,filenames,titles,Qs):
    eps = [projected[0:len_ep1],projected[len_ep1:len_ep1+len_ep2],projected[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3]]
    cmap = mpl.colormaps['Set1']
    colors = cmap(np.linspace(0,0.5,n_mass))
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    for i in range(n_mass):
        projected = eps[i]
        ax.plot(projected[:,0],projected[:,1],projected[:,2],color=colors[i],linewidth=1,label='Mass '+str(i+1))

    ax.view_init(elev=30.,azim=45)

    leg = plt.legend(fontsize=21,loc='center left', bbox_to_anchor=(0.7, 0.9))
    for line in leg.get_lines():
        line.set_linewidth(4.0)
    plt.xlabel('Proj. 1',fontsize=21,labelpad=10)
    plt.ylabel('Proj. 2',fontsize=21,labelpad=10)
    ax.set_zlabel('Proj. 3',fontsize=21,labelpad=10)
    plt.title(title,fontsize=21)
    ax.text2D(0.01,0.96, ha='left', va='top', transform=ax.transAxes, s='Q = %s'%np.round(q,3), color='black', 
        bbox=dict(facecolor='none', edgecolor='grey', boxstyle='round',pad=0.3),fontsize=21)
    
    ax.locator_params(axis='x', nbins=5)
    ax.locator_params(axis='y', nbins=5)
    ax.locator_params(axis='z', nbins=5)
    font = {'size': 19}
    plt.xticks(fontsize=font['size'])
    plt.yticks(fontsize=font['size'])
    ax.tick_params('z', labelsize=font['size'])
    plt.show()

5. Plot the tangling metric for each layer (+obs, +acts) through time

In [ ]:
for Qs, title, filename in zip([[q_obs_ep1,q_obs_ep2,q_obs_ep3],[q_lstm_ep1,q_lstm_ep2,q_lstm_ep3],[q_acts_ep1,q_acts_ep2,q_acts_ep3]],['Q Observations','Q LSTM','Q Actions'],['qobs-t','qlstm-t','qacts-t']):
    figure = plt.figure()
    for i in range(len(Qs)) :
        plt.plot(np.arange(0,len(Qs[i]),1),Qs[i],linewidth=1,label='Mass %s'%i,alpha=0.8)
    plt.ylabel(title,fontsize=21)
    plt.xlabel('Time step',fontsize=21)
    plt.xticks(fontsize=19,rotation=45)
    plt.yticks(fontsize=19)
    plt.subplots_adjust(left=0.17,bottom=0.2)
    plt.show()

6. Compare _a_. Q observations with Q actions and _b_. Q LSTM with Q actions, for all the time instances

In [ ]:
obs_trans_per_mass = [obs_trans[0:len_ep1],obs_trans[len_ep1:len_ep1+len_ep2],obs_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3]]
lstm_trans_per_mass = [lstm_trans[0:len_ep1],lstm_trans[len_ep1:len_ep1+len_ep2],lstm_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3]]
acts_trans_per_mass = [acts_trans[0:len_ep1],acts_trans[len_ep1:len_ep1+len_ep2],acts_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3]]

for trans,title,filename in zip([obs_trans_per_mass,lstm_trans_per_mass],['Q observations','Q LSTM'],['obs-acts','lstm-acts']):
    figure = plt.figure()
    for i in range(n_mass):
        plt.scatter(measure_tangling(acts_trans_per_mass[i]),measure_tangling(trans[i]),s=7,label='Mass %s'%i,alpha=0.8)
    plt.ylabel(title,fontsize=21)
    plt.xlabel('Q actions',fontsize=21)
    plt.xticks(fontsize=19)
    plt.yticks(fontsize=19)
    lgnd = plt.legend(fontsize=21,loc='center left', bbox_to_anchor=(0.7, 0.7))
    lgnd.legendHandles[0]._sizes = [40]
    lgnd.legendHandles[1]._sizes = [40]
    lgnd.legendHandles[2]._sizes = [40]
    plt.subplots_adjust(left=0.15,bottom=0.2)
    pt = (0, 0)
    plt.axline(pt, slope=1, color='black')
    plt.axis('square')
    plt.show()

### B. PCA
1. Load trajectories of observations, LSTM, Linear 1, Linear 2 and actions for 3 different masses (same data as in section A)

In [ ]:
# Load the file from Basecamp : 'layers_mass'
layers = pickle.load(open('/home/ingster/Bureau/SIL-BigResults/layers_mass','rb'))
n_mass = 3

3. a. Apply PCA for each layer (+obs, +acts) across masses\
b. Compute the tangling metric per layer and per mass\
c. Compute the global tangling metric per layer (tangling metric averaged over mass and time)\
\
__Remark__. The percentage of explained variance accounted using 25 PCs is :
- 98% for the observations,
- 70% for the LSTM output,
- 87% for the Linear 1 and Linear 2 outputs,
- 94% for the actions.

In [ ]:
len_ep1 = len(layers[0])
len_ep2 = len(layers[1])
len_ep3 = len(layers[2])

layers = [item for sublist in layers for item in sublist] 

n_comp = 25
pca = PCA(n_components=n_comp)

obs = np.array([d['observation'] for d in layers])
obs_trans = pca.fit_transform(obs)
q_obs_ep1 = measure_tangling(obs_trans[0:len_ep1])
q_obs_ep2 = measure_tangling(obs_trans[len_ep1:len_ep1+len_ep2])
q_obs_ep3 = measure_tangling(obs_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_obs = np.mean(np.mean(q_obs_ep1)+np.mean(q_obs_ep2)+np.mean(q_obs_ep3))

lstm = np.array([d['LSTM hidden state'] for d in layers])
lstm_trans = pca.fit_transform(lstm)
q_lstm_ep1 = measure_tangling(lstm_trans[0:len_ep1])
q_lstm_ep2 = measure_tangling(lstm_trans[len_ep1:len_ep1+len_ep2])
q_lstm_ep3 = measure_tangling(lstm_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_lstm = np.mean(np.mean(q_lstm_ep1)+np.mean(q_lstm_ep2)+np.mean(q_lstm_ep3))

l1 = np.array([d['Linear layer 1'] for d in layers])
l1_trans = pca.fit_transform(l1)

q_l1_ep1 = measure_tangling(l1_trans[0:len_ep1])
q_l1_ep2 = measure_tangling(l1_trans[len_ep1:len_ep1+len_ep2])
q_l1_ep3 = measure_tangling(l1_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_l1 = np.mean(np.mean(q_l1_ep1)+np.mean(q_l1_ep2)+np.mean(q_l1_ep3))

l2 = np.array([d['Linear layer 2'] for d in layers])
l2_trans = pca.fit_transform(l2)
q_l2_ep1 = measure_tangling(l2_trans[0:len_ep1])
q_l2_ep2 = measure_tangling(l2_trans[len_ep1:len_ep1+len_ep2])
q_l2_ep3 = measure_tangling(l2_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_l2 = np.mean(np.mean(q_l2_ep1)+np.mean(q_l2_ep2)+np.mean(q_l2_ep3))

acts = np.array([d['actions'] for d in layers])
acts_trans = pca.fit_transform(acts)
q_acts_ep1 = measure_tangling(acts_trans[0:len_ep1])
q_acts_ep2 = measure_tangling(acts_trans[len_ep1:len_ep1+len_ep2])
q_acts_ep3 = measure_tangling(acts_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3])
Q_acts = np.mean(np.mean(q_acts_ep1)+np.mean(q_acts_ep2)+np.mean(q_acts_ep3))

4. Plot the tangling metric for each layer (+obs, +acts) through time

In [ ]:
for Qs, title, filename in zip([[q_obs_ep1,q_obs_ep2,q_obs_ep3],[q_lstm_ep1,q_lstm_ep2,q_lstm_ep3],[q_acts_ep1,q_acts_ep2,q_acts_ep3]],['Q Observations','Q LSTM','Q Actions'],['qobs-t','qlstm-t','qacts-t']):
    figure = plt.figure()
    for i in range(len(Qs)) :
        plt.plot(np.arange(0,len(Qs[i]),1),Qs[i],label='Mass %s'%i,linewidth=1,alpha=0.8)
    plt.ylabel(title,fontsize=21)
    plt.xlabel('Time step',fontsize=21)
    plt.xticks(fontsize=19,rotation=45)
    plt.yticks(fontsize=19)
    plt.subplots_adjust(left=0.17,bottom=0.2)
    plt.show()


5. Compare _a_. Q observations with Q actions and _b_. Q LSTM with Q actions, for all the time instances

In [ ]:
obs_trans_per_mass = [obs_trans[0:len_ep1],obs_trans[len_ep1:len_ep1+len_ep2],obs_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3]]
lstm_trans_per_mass = [lstm_trans[0:len_ep1],lstm_trans[len_ep1:len_ep1+len_ep2],lstm_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3]]
acts_trans_per_mass = [acts_trans[0:len_ep1],acts_trans[len_ep1:len_ep1+len_ep2],acts_trans[len_ep1+len_ep2:len_ep1+len_ep2+len_ep3]]

for trans, title,filename in zip([obs_trans_per_mass,lstm_trans_per_mass],['Q observations','Q LSTM'],['obs-acts','lstm-acts']):
    figure = plt.figure()
    for i in range(n_mass):
        plt.scatter(measure_tangling(acts_trans_per_mass[i]),measure_tangling(trans[i]),s=7,label='Mass %s'%i,alpha=0.8)
    plt.ylabel(title,fontsize=21)
    plt.xlabel('Q actions',fontsize=21)
    plt.xticks(fontsize=19)
    plt.yticks(fontsize=19)
    lgnd = plt.legend(fontsize=21,loc='center left', bbox_to_anchor=(0.7, 0.7))
    lgnd.legendHandles[0]._sizes = [40]
    lgnd.legendHandles[1]._sizes = [40]
    lgnd.legendHandles[2]._sizes = [40]
    plt.subplots_adjust(left=0.15,bottom=0.2)
    pt = (0, 0)
    plt.axline(pt, slope=1, color='black')
    plt.axis('square')
    plt.show()

### C. Full space
1. Load trajectories of observations, LSTM, Linear 1, Linear 2 and actions for 3 different masses (same data as in section A and B)

In [ ]:
# Load the file from Basecamp : 'layers_mass'
layers = pickle.load(open('/home/ingster/Bureau/SIL-BigResults/layers_mass','rb'))
n_mass = 3

2. a. Compute the tangling metric per layer and per mass\
b. Compute the global tangling metric per layer (tangling metric averaged over mass and time)\

In [ ]:
obs_ep1 = [d['observation'] for d in layers[0]]
obs_ep2 = [d['observation'] for d in layers[1]]
obs_ep3 = [d['observation'] for d in layers[2]]

q_obs_ep1 = measure_tangling(obs_ep1)
q_obs_ep2 = measure_tangling(obs_ep2)
q_obs_ep3 = measure_tangling(obs_ep3)
Q_obs = np.mean(np.mean(q_obs_ep1)+np.mean(q_obs_ep2)+np.mean(q_obs_ep3))

lstm_ep1 = np.array([d['LSTM hidden state'] for d in layers[0]])
lstm_ep2 = np.array([d['LSTM hidden state'] for d in layers[1]])
lstm_ep3 = np.array([d['LSTM hidden state'] for d in layers[2]])

q_lstm_ep1 = measure_tangling(lstm_ep1)
q_lstm_ep2 = measure_tangling(lstm_ep2)
q_lstm_ep3 = measure_tangling(lstm_ep3)
Q_lstm = np.mean(np.mean(q_lstm_ep1)+np.mean(q_lstm_ep2)+np.mean(q_lstm_ep3))

l1_ep1 = np.array([d['Linear layer 1'] for d in layers[0]])
l1_ep2 = np.array([d['Linear layer 1'] for d in layers[1]])
l1_ep3 = np.array([d['Linear layer 1'] for d in layers[2]])

q_l1_ep1 = measure_tangling(l1_ep1)
q_l1_ep2 = measure_tangling(l1_ep2)
q_l1_ep3 = measure_tangling(l1_ep3)
Q_l1 = np.mean(np.mean(q_l1_ep1)+np.mean(q_l1_ep2)+np.mean(q_l1_ep3))

l2_ep1 = np.array([d['Linear layer 2'] for d in layers[0]])
l2_ep2 = np.array([d['Linear layer 2'] for d in layers[1]])
l2_ep3 = np.array([d['Linear layer 2'] for d in layers[2]])

q_l2_ep1 = measure_tangling(l2_ep1)
q_l2_ep2 = measure_tangling(l2_ep2)
q_l2_ep3 = measure_tangling(l2_ep3)
Q_l2 = np.mean(np.mean(q_l2_ep1)+np.mean(q_l2_ep2)+np.mean(q_l2_ep3))

acts_ep1 = np.array([d['actions'] for d in layers[0]])
acts_ep2 = np.array([d['actions'] for d in layers[1]])
acts_ep3 = np.array([d['actions'] for d in layers[2]])

q_acts_ep1 = measure_tangling(acts_ep1)
q_acts_ep2 = measure_tangling(acts_ep2)
q_acts_ep3 = measure_tangling(acts_ep3)
Q_acts = np.mean(np.mean(q_acts_ep1)+np.mean(q_acts_ep2)+np.mean(q_acts_ep3))

3. Plot the tangling metric for each layer (+obs, +acts) through time

In [ ]:
for Qs, title, filename in zip([[q_obs_ep1,q_obs_ep2,q_obs_ep3],[q_lstm_ep1,q_lstm_ep2,q_lstm_ep3],[q_acts_ep1,q_acts_ep2,q_acts_ep3]],['Q Observations','Q LSTM','Q Actions'],['qobs-t','qlstm-t','qacts-t']):
    figure = plt.figure()
    for i in range(len(Qs)) :
        plt.plot(np.arange(0,len(Qs[i]),1),Qs[i],linewidth=1,label='Mass %s'%i,alpha=0.8)
    plt.ylabel(title,fontsize=21)
    plt.xlabel('Time step',fontsize=21)
    plt.xticks(fontsize=19,rotation=45)
    plt.yticks(fontsize=19)
    plt.subplots_adjust(left=0.17,bottom=0.2)
    plt.show()

4. Compare _a_. Q observations with Q actions and _b_. Q LSTM with Q actions, for all the time instances

In [ ]:
obs_per_mass = [obs_ep1,obs_ep2,obs_ep3]
lstm_per_mass = [lstm_ep1,lstm_ep2,lstm_ep3]
acts_per_mass = [acts_ep1,acts_ep2,acts_ep3]

for trans, title,filename in zip([obs_per_mass,lstm_per_mass],['Q observations','Q LSTM'],['obs-acts','lstm-acts']):
    figure = plt.figure()
    for i in range(n_mass):
        plt.scatter(measure_tangling(acts_per_mass[i]),measure_tangling(trans[i]),s=7,label='Mass %s'%i,alpha=0.8)
    plt.ylabel(title,fontsize=21)
    plt.xlabel('Q actions',fontsize=21)
    plt.xticks(fontsize=19)
    plt.yticks(fontsize=19)
    lgnd = plt.legend(fontsize=21,loc='center left', bbox_to_anchor=(0.7, 0.7))
    lgnd.legendHandles[0]._sizes = [40]
    lgnd.legendHandles[1]._sizes = [40]
    lgnd.legendHandles[2]._sizes = [40]
    plt.subplots_adjust(left=0.15,bottom=0.2)
    pt = (0, 0)
    plt.axline(pt, slope=1, color='black')
    plt.axis('square')
    plt.show()